<a href="https://colab.research.google.com/github/tungsontran/Synthetic-Medical-Transcript-Classifier/blob/main/Transcript_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers
import os
import requests
import json

import numpy as np
import pandas as pd

from google.colab import drive
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
!pip install sentence_transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Path to the CSV file
file_path = '/content/drive/My Drive/Colab Notebooks/Dataset/mtsamples.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(file_path, index_col=0)
# Clean data a lil bit
for _,row in df.iterrows():
  row['medical_specialty']=row['medical_specialty'][1:]
# Data is a list of dict
data = json.loads(df.to_json(orient='records',indent=2))

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from typing import List, Dict, Any
def encode_data(data: List[Dict[str,Any]]) -> List[Dict[str,Any]]:
  data_encoded = []
  for sample in data:
    sample_encoded = {}
    sample_encoded['Label'] = sample['medical_specialty']
    sample = json.dumps(sample)
    sample_encoded['Document'] = model.encode(sample).tolist()
    data_encoded.append(sample_encoded)
  return data_encoded

In [ ]:
mtsamples_encoded = encode_data(data)

with open('/content/drive/My Drive/Colab Notebooks/Dataset/mtsamples_encoded.json', 'w') as json_file:
    json.dump(mtsamples_encoded, json_file)

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Dataset/mtsamples_encoded.json') as json_file:
    mtsamples_encoded = json.load(json_file)

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Dataset/mtsamples_synthetic.json') as json_file:
    mtsamples_synthetic = json.load(json_file)

mtsamples_synthetic_encoded = encode_data(mtsamples_synthetic)

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report

def knn_classification_test(data, k, metric='euclidean', target_class=None, n_splits=5, random_state=42):
    X = np.array([d['Document'] for d in data])
    y = np.array([d['Label'] for d in data])

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    accuracies = []
    recalls = []
    precisions = []
    f1_scores = []
    class_reports = []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Create and fit KNeighborsClassifier
        if metric == 'euclidean':
            knn_classifier = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
        elif metric == 'cosine':
            knn_classifier = KNeighborsClassifier(n_neighbors=k, metric='cosine')
        knn_classifier.fit(X_train, y_train)

        # Predict classes for the test set
        y_pred = knn_classifier.predict(X_test)

        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='weighted',zero_division=0.0)
        precision = precision_score(y_test, y_pred, average='weighted',zero_division=0.0)
        f1 = f1_score(y_test, y_pred, average='weighted',zero_division=0.0)

        accuracies.append(accuracy)
        recalls.append(recall)
        precisions.append(precision)
        f1_scores.append(f1)

        # If target_class is specified, compute metrics for that specific class
        if target_class:
            class_report = classification_report(y_test, y_pred, labels=target_class, target_names=target_class, zero_division=0.0)
            return accuracy, recall, precision, f1, class_report

    # Calculate average metrics across folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_recall = sum(recalls) / len(recalls)
    avg_precision = sum(precisions) / len(precisions)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    if target_class:
        avg_class_report = "\n\n".join(class_reports)
        return avg_accuracy, avg_recall, avg_precision, avg_f1, avg_class_report

    return avg_accuracy, avg_recall, avg_precision, avg_f1

In [ ]:
target_class = ['Orthopedic']
accuracy, recall, precision, f1, class_report = knn_classification_test(mtsamples_encoded, 10, metric='euclidean', target_class=target_class)
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall:, {recall:.4f}")
print(f"Precision:, {precision:.4f}")
print(f"F1-score:, {f1:.4f}")
print("Classification Report for", target_class, "class:")
print(class_report)

Accuracy: 0.3330
Recall:, 0.3330
Precision:, 0.3233
F1-score:, 0.3151
Classification Report for Orthopedic class:
              precision    recall  f1-score   support

  Orthopedic       0.34      0.51      0.41        71

   micro avg       0.34      0.51      0.41        71
   macro avg       0.34      0.51      0.41        71
weighted avg       0.34      0.51      0.41        71



In [ ]:
def knn_classification(input, data, k, metric='euclidean', target_class=None):
    # Extract embeddings and labels from data
    X = [np.array(d['Document']) for d in data]
    y = [d['Label'] for d in data]

    # Create and fit KNeighborsClassifier
    if metric == 'euclidean':
        knn_classifier = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    elif metric == 'cosine':
        knn_classifier = KNeighborsClassifier(n_neighbors=k, metric='cosine')
    knn_classifier.fit(X, y)

    # Predict class for the sample text
    if isinstance(input,dict):
      predicted_class = knn_classifier.predict([input])[0]
      return predicted_class
    elif isinstance(input,list):
      X_test = []
      y_test = []
      for sample in input:
          X_test.append(sample['Document'])
          y_test.append(sample['Label'])

      y_pred = knn_classifier.predict(X_test)

      # Evaluate the model
      accuracy = accuracy_score(y_test, y_pred)
      recall = recall_score(y_test, y_pred, average='weighted',zero_division=0.0)
      precision = precision_score(y_test, y_pred, average='weighted',zero_division=0.0)
      f1 = f1_score(y_test, y_pred, average='weighted',zero_division=0.0)

      # If target_class is specified, compute metrics for that specific class
      if target_class:
          class_report = classification_report(y_test, y_pred, labels=target_class, target_names=target_class,zero_division=0.0)
          return accuracy, recall, precision, f1, class_report

      return accuracy, recall, precision, f1

In [ ]:
target_class = ['Cardiovascular / Pulmonary','Surgery']
accuracy, recall, precision, f1, class_report = knn_classification(mtsamples_synthetic_encoded,mtsamples_encoded,5,'euclidean',target_class)
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall:, {recall:.4f}")
print(f"Precision:, {precision:.4f}")
print(f"F1-score:, {f1:.4f}")
print("Classification Report for", target_class, "class:")
print(class_report)

Accuracy: 0.6154
Recall:, 0.6154
Precision:, 1.0000
F1-score:, 0.7619
Classification Report for ['Cardiovascular / Pulmonary', 'Surgery'] class:
                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       1.00      0.62      0.76        13
                   Surgery       0.00      0.00      0.00         0

                 micro avg       1.00      0.62      0.76        13
                 macro avg       0.50      0.31      0.38        13
              weighted avg       1.00      0.62      0.76        13



In [ ]:
sample = {
    "description": "Patient presents with symptoms of shortness of breath and chest pain. Diagnostic tests reveal the presence of a heart defect and pulmonary hypertension. Treatment plan includes medications and lifestyle modifications.",
    "medical_specialty": "Cardiovascular / Pulmonary",
    "sample_name": "Cardiovascular/Pulmonary Case",
    "transcription": "Patient is a 45-year-old male presenting with symptoms of shortness of breath and chest pain. The patient reports that these symptoms have been present for the past two weeks. Upon further questioning, the patient states that the symptoms worsen with exertion and are accompanied by fatigue. The patient denies any other symptoms. Physical exam is unremarkable. Initial labs reveal normal CBC, electrolytes, and renal and liver function. ECG shows sinus rhythm with no significant findings. Chest X-ray is within normal limits. The patient undergoes pulmonary function testing, which reveals a forced expiratory volume in one second (FEV1) of 80% and a forced vital capacity (FVC) of 90%. This indicates restrictive lung disease. The patient is admitted for further evaluation and monitoring. On hospital day 2, the patient undergoes a right heart catheterization, which reveals a pulmonary artery pressure of 55/25 mm Hg and a pulmonary capillary wedge pressure of 14 mm Hg. This indicates the presence of pulmonary hypertension. Echocardiography confirms the presence of a ventricular septal defect (VSD) with left-to-right shunt flow. The patient is started on medications to manage his symptoms and to decrease the likelihood of further complications. The patient is advised to make lifestyle modifications to manage his condition, including avoiding overexertion and maintaining a healthy weight.",
    "keywords": [
        "heart defect",
        "pulmonary hypertension",
        "shortness of breath",
        "chest pain",
        "restrictive lung disease",
        "echocardiography",
        "right heart catheterization"
    ]
}
sample = json.dumps(sample)
sample_encoded = model.encode(sample)
knn_classification(sample_encoded, outputs, k=10, metric='cosine')

'Cardiovascular / Pulmonary'

In [ ]:
sample2 = {
    "description": "Summary of patient's medical history and physical examination.",
    "medical_specialty": "Consult - History and Phy.",
    "sample_name": "Consult - History and Phy.",
    "transcription": "Patient is a 65-year-old male presented with a 2-week history of shortness of breath. He denies any chest pain, palpitations, or syncope. He reports no recent travel or sick contacts. He is a retired construction worker and smokes one pack per day for 40 years. He has a significant past medical history including hypertension, type 2 diabetes mellitus, and hyperlipidemia, all well-controlled on medications. He underwent radical prostatectomy for prostate cancer 5 years ago and underwent left lower extremity revascularization for claudication 2 years ago. He takes aspirin 81 mg daily, metoprolol, lisinopril, hydrochlorothiazide, glipizide, atorvastatin, and acetaminophen with codeine as needed for pain. His home medications were reviewed and refills were provided. Vital signs revealed blood pressure of 145/85 mm Hg, pulse 72 beats/min, respiratory rate 18 breaths/min, and oxygen saturation 94% on room air. He was in no acute distress. General examination showed well-developed male with a BMI of 28. His neck was supple with no lymphadenopathy. His chest was unremarkable with no signs of trauma, clubbing, or cyanosis. His cardiac examination revealed a regular rate and rhythm without any murmurs, rubs, or gallops. His abdomen was benign without any tenderness or hepatosplenomegaly. His extremities revealed no edema or varicose veins. His neurologic examination was unremarkable. Based on his past medical history and physical examination, it is suggested that his shortness of breath is most likely due to his underlying chronic obstructive pulmonary disease (COPD) and/or heart failure (HF). Pulmonary function test and echocardiogram were recommended. His lungs were clear to auscultation and percussion. He had no rales, rhonchi, or wheezes. His heart sounds were normal without any murmurs, rubs, or gallops. His extremities revealed no edema. His skin was without any rashes, lesions, or clubbing. His extremities revealed 2+ lower extremity edema, mostly at the ankles. His neurovascular status was intact. His reflexes were 2+ and symmetric. He had no focal weakness, sensory deficits, or atrophy. His fundi were benign without any hemorrhages or exudates. He had no costovertebral angle tenderness.",
    "keywords": [
        "shortness of breath",
        "hypertension",
        "diabetes mellitus",
        "hyperlipidemia",
        "prostate cancer",
        "left lower extremity revascularization",
        "aspirin",
        "metoprolol",
        "lisinopril",
        "hydrochlorothiazide",
        "glipizide",
        "atorvastatin",
        "acetaminophen",
        "codeine",
        "BMI",
        "COPD",
        "heart failure",
        "pulmonary function test",
        "echocardiogram",
        "rahonchi",
        "wheezes",
        "edema",
        "neurovascular status"
    ]
}
}
sample2 = json.dumps(sample2)
sample2_encoded = model.encode(sample2)
knn_classification(sample2_encoded, outputs, k=10, metric='cosine')

'Consult - History and Phy.'

In [ ]:
sample3 = {
    "description": "Patient with a history of coronary artery disease and chronic obstructive pulmonary disease presents with shortness of breath and chest pain. Echocardiogram reveals left ventricular dysfunction and pulmonary arterial hypertension. Patient is started on diuretics, beta-blockers, and oxygen therapy.",
    "medical_specialty": "Cardiovascular / Pulmonary",
    "sample_name": "72-year-old male with coronary artery disease and COPD presenting with dyspnea and chest pain.",
    "transcription": "Patient is a 72-year-old male with a past medical history significant for coronary artery disease and chronic obstructive pulmonary disease, presenting with complaints of shortness of breath and chest pain. The patient reports that his symptoms have been ongoing for several weeks and are worse with exertion. He denies any syncope or presyncope. The patient's vital signs are significant for a blood pressure of 130/80 mm Hg, a heart rate of 85 beats per minute, and an oxygen saturation of 88% on room air. The patient appears comfortable and in no acute distress. Physical examination is significant for crackles in the right lower lung field. The patient's cardiac exam is notable for a regular rate and rhythm, without murmurs, rubs, or gallops. The patient's extremities are warm and well-perfused. The patient's abdomen is benign. The patient's neurologic exam is intact. The patient's skin is without rash or edema. Based on the patient's history and examination, laboratory studies are ordered, including a complete blood count, comprehensive metabolic panel, lipid panel, troponin I, brain natriuretic peptide, and d-dimer. Echocardiography is also ordered to further evaluate the patient's cardiac function. The patient is admitted to the hospital for further evaluation and management.",
    "keywords": [
        "coronary artery disease",
        "chronic obstructive pulmonary disease",
        "dyspnea",
        "chest pain",
        "echocardiogram",
        "left ventricular dysfunction",
        "pulmonary arterial hypertension",
        "diuretics",
        "beta-blockers",
        "oxygen therapy"
    ]
}
sample3 = json.dumps(sample3)
sample3_encoded = model.encode(sample3)
knn_classification(sample3_encoded, outputs, k=10, metric='cosine')

'Cardiovascular / Pulmonary'